In [ ]:
from dotenv import load_dotenv, find_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.chat_models import ChatOllama
import requests

load_dotenv(find_dotenv())

In [ ]:
def fetch_model_names(base_url):
    api_endpoint = f"{base_url}/api/tags"
    try:
        response = requests.get(api_endpoint)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        model_names = [model["name"] for model in data.get("models", [])]
        return model_names
    except requests.RequestException as e:
        return str(e)


base_url = "https://ollama.junzis.com"
model_names = fetch_model_names(base_url)
model = model_names[0]
model

In [ ]:
import sys

sys.path.append("../../bluesky")
from bluesky.network.client import Client

client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()

In [ ]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

import time
from contextlib import contextmanager
from io import StringIO
from langchain_core.tools import tool as langchain_tool


# Step 1 (Optional): Add a custom tool named `my_image_gen`.
@register_tool("my_image_gen")
class MyImageGen(BaseTool):
    # The `description` tells the agent the functionality of this tool.
    description = "AI painting (image generation) service, input text description, and return the image URL drawn based on text information."
    # The `parameters` tell the agent what input parameters the tool has.
    parameters = [
        {
            "name": "prompt",
            "type": "string",
            "description": "Detailed description of the desired image content, in English",
            "required": True,
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        # `params` are the arguments generated by the LLM agent.
        prompt = json5.loads(params)["prompt"]
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {"image_url": f"https://image.pollinations.ai/prompt/{prompt}"},
            ensure_ascii=False,
        )


@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def receive_bluesky_output():
    complete_output = ""
    empty_output_count = 0  # Track consecutive empty outputs

    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()

        # Check if the current output is empty
        if not new_output.strip():
            empty_output_count += 1  # Increment counter for empty outputs
        else:
            empty_output_count = 0  # Reset counter if output is not empty
            complete_output += new_output  # Add non-empty output to complete output

        # If there are two consecutive empty outputs, break the loop
        if empty_output_count >= 5:
            break

    # It's assumed you want to keep the last update outside the loop
    client.update()

    return complete_output


@register_tool("get_conflict_info")
class Conflict(BaseTool):
    # The `description` tells the agent the functionality of this tool.
    description = "Get conflict information between aircraft pairs. It gives you Time to Closest Point of Approach (TCPA), Quadrantal Direction (QDR), separation distance, Closest Point of Approach distance (DCPA), and Time of Loss of Separation (tLOS)."
    # The `parameters` tell the agent what input parameters the tool has.
    parameters = [
        {
            "name": "command",
            "type": "string",
            "description": "Command to get conflict information between aircraft pairs. Default is 'SHOWTCPA'",
            "required": True,
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        # `params` are the arguments generated by the LLM agent.
        command = json5.loads(params)["command"]
        command = urllib.parse.quote(command)
        client.send_event(b"STACK", "SHOWTCPA")
        time.sleep(0.8)
        sim_output = receive_bluesky_output()
        return json5.dumps(
            {"conflict_info": sim_output},
            ensure_ascii=False,
        )




In [ ]:
@register_tool("get_aircraft_info")
class Conflict(BaseTool):
    # The `description` tells the agent the functionality of this tool.
    description = "Get all aircraft information in the simulation. It gives you aircraft ID, altitude, heading, speed, and position."
    # The `parameters` tell the agent what input parameters the tool has.
    parameters = [
        {
            "name": "command",
            "type": "string",
            "description": "Command to get aircraft information. Default is 'GETACIDS'",
            "required": True,
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        # `params` are the arguments generated by the LLM agent.
        command = json5.loads(params)["command"]
        command = urllib.parse.quote(command)
        client.send_event(b"STACK", "GETACIDS")
        time.sleep(0.8)
        sim_output = receive_bluesky_output()
        return json5.dumps(
            {"aircraft_info": sim_output},
            ensure_ascii=False,
        )

In [ ]:
# Step 2: Configure the LLM you are using.
llm_cfg = {
    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    "model": "qwen2:72b",
    "model_server": "https://ollama.junzis.com/v1",  # base_url, also known as api_base
    "api_key": "EMPTY",
    # (Optional) LLM hyperparameters for generation:
    # "generate_cfg": {"top_p": 0.8},
}

# Step 3: Create an agent. Here we use the `Assistant` agent as an example, which is capable of using tools and reading files.
system_instruction = """You are a helpful assistant."""
tools = [
    "get_conflict_info",
    "get_aircraft_info",
]  # `code_interpreter` is a built-in tool for executing code.
bot = Assistant(
    llm=llm_cfg, system_message=system_instruction, function_list=tools
)
# Step 4: Run the agent as a chatbot.
# messages = []  # This stores the chat history.
# while True:
#     # For example, enter the query "draw a dog and rotate it 90 degrees".
#     query = input("user query: ")
#     # Append the user query to the chat history.
#     messages.append({"role": "user", "content": query})
#     response = []
#     for response in bot.run(messages=messages):
#         # Streaming output.
#         print("bot response:")
#         pprint.pprint(response, indent=2)
#     # Append the bot responses to the chat history.
#     messages.extend(response)

In [ ]:
def run_bot_and_collect_messages(bot, messages):
    output = bot.run(messages=messages)
    final_responses = []
    try:
        while True:
            response = next(output)
            final_responses.extend(response)
    except StopIteration:
        pass  # The generator has no more output
    return final_responses

In [ ]:
responses = run_bot_and_collect_messages(bot, [{"role": "user", "content": "please check if there are any conflicts between aircrafts"}])

In [ ]:
messages=[
    {
        "role": "user",
        "content": "please check if there are any conflicts between aircrafts and if so give me a conflict resolution plan according to ICAO standards. ICAO standards: minimum 5 nm lateral separation or 1000 ft vertical separation",
    }
]

In [ ]:
task = "please check if there are any conflicts between aircrafts and if so give me a conflict resolution plan according to ICAO standards. ICAO standards: minimum 5 nm lateral separation or 1000 ft vertical separation"

In [ ]:
messages = []
while True:
    # For example, enter the query "draw a dog and rotate it 90 degrees".
    # query = input("user query: ")
    query = "please check if there are any conflicts between aircrafts and if so give me a conflict resolution plan according to ICAO standards. ICAO standards: minimum 5 nm lateral separation or 1000 ft vertical separation"
    # Append the user query to the chat history.
    messages.append({"role": "user", "content": query})
    response = []
    for response in bot.run(messages=messages):
        # Streaming output.
        print("bot response:")
        pprint.pprint(response, indent=2)
    # Append the bot responses to the chat history.
    messages.extend(response)

In [ ]:
*_, last = bot.run(
    messages=[
        {
            "role": "user",
            "content": "tell me a joke",
        }
    ]
)

In [ ]:
for response in bot.run(messages=messages):
    print(response)